Using Spark SQL in Spark Applications

In [0]:
from pyspark.sql import SparkSession



Basic Query Examples

In [0]:
#Creamos la sesion Spark
spark = (SparkSession.builder.appName("SparkSqlEjemplo").getOrCreate())
schema = "`fecha` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"
vuelosDF=(spark.read
    .schema(schema)
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .csv("dbfs:/FileStore/FileStore/cap4/departuredelays.csv"))
vuelosDF.createOrReplaceTempView("vuelos")
         




#df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/FileStore/cap4/departuredelays.csv")
#df1.createOrReplaceTempView("vuelos")


In [0]:
#Creamos Schema



Observemos el contenido de la tabla

In [0]:
vuelosDF.printSchema()

root
 |-- fecha: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [0]:
spark.sql("""SELECT * from vuelos""").show(10)


+--------+-----+--------+------+-----------+
|   fecha|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [0]:
#vuelos cuya distancia es mayor a 1000 millas
spark.sql("SELECT distance from vuelos where distance > 1000").show(10)

+--------+
|distance|
+--------+
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
+--------+
only showing top 10 rows



In [0]:
# In Python
from pyspark.sql.functions import col
(vuelosDF.select("distance").where(col("distance") > 1000)).show(10)


+--------+
|distance|
+--------+
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
|    1586|
+--------+
only showing top 10 rows



Encuentre todos los vuelos entre San Francisco (SFO) y Chicago (ORD) con al menos dos horas de retraso:

In [0]:
spark.sql("SELECT * from vuelos where delay >= 120 and origin = 'SFO' and destination = 'ORD' order by delay DESC").show(10)

+--------+-----+--------+------+-----------+
|   fecha|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02190925| 1638|    1604|   SFO|        ORD|
|01031755|  396|    1604|   SFO|        ORD|
|01022330|  326|    1604|   SFO|        ORD|
|01051205|  320|    1604|   SFO|        ORD|
|01190925|  297|    1604|   SFO|        ORD|
|02171115|  296|    1604|   SFO|        ORD|
|01071040|  279|    1604|   SFO|        ORD|
|01051550|  274|    1604|   SFO|        ORD|
|03120730|  266|    1604|   SFO|        ORD|
|01261104|  258|    1604|   SFO|        ORD|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [0]:
df1.select("*").show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [0]:

string= vista[0]
print(string)


Column<'date'>


In [0]:
def to_date_format_udf(d_str):
l = [char for char in d_str]
return "".join(l[0:2]) + "/" + "".join(l[2:4]) + " " + " " +"".join(l[4:6]) + ":" + "".join(l[6:])




s


In [0]:
#convertir el dato date

#sua= df1.withColumn("timestamp",to_timestamp("input_timestamp"))
#final_string = my_string[:index] + 'you ' + my_string[index:]
#sua.printSchema()


Label all US flights, regardless of origin and destination,with an indication of the delays they experienced: Very Long Delays (> 6 hours),Long Delays (2–6 hours), etc

In [0]:
vuelosDF.select("delay", "origin", "destination").show(10).withColumn("Flights_Delays",when(col("delay") >360 , "Very Long Delays")


  File "<command-1395344244104865>", line 1
    vuelosDF.select("delay", "origin", "destination").show(10).withColumn("Flights_Delays",when(col("delay") >360 , "Very Long Delays")
                                                                                                                                       ^
SyntaxError: unexpected EOF while parsing


In [0]:
spark.sql("""SELECT delay, origin, destination,
 CASE
 WHEN delay > 360 THEN 'Very Long Delays'
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
 WHEN delay = 0 THEN 'No Delays'
 ELSE 'Early'
 END AS Retraso_del_vuelo
 FROM vuelos""").show(10)


+-----+------+-----------+-----------------+
|delay|origin|destination|Retraso_del_vuelo|
+-----+------+-----------+-----------------+
|    6|   ABE|        ATL| Tolerable Delays|
|   -8|   ABE|        DTW|            Early|
|   -2|   ABE|        ATL|            Early|
|   -4|   ABE|        ATL|            Early|
|   -4|   ABE|        ATL|            Early|
|    0|   ABE|        ATL|        No Delays|
|   10|   ABE|        ATL| Tolerable Delays|
|   28|   ABE|        ATL| Tolerable Delays|
|   88|   ABE|        ATL|     Short Delays|
|    9|   ABE|        ATL| Tolerable Delays|
+-----+------+-----------+-----------------+
only showing top 10 rows



Creando y usando BD

Managed table

In [0]:
spark.sql("Create table tabla_vuelos (fecha STRING, retraso INT, distancia INT, origen STRING, destino STRING)")

Out[15]: DataFrame[]

Lo mismo pero usando API de DataFrame

In [0]:
# In Python
# Path to our US flight delays CSV file 
csv_file = "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
# Schema as defined in the preceding example
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")


Unmanaged

In [0]:
(flights_df
 .write
 .option("path", "dbfs:/FileStore/tutorial/data")
 .saveAsTable("us_delay_flights_tbl"))


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-513246272696510> in <module>
----> 1 (flights_df
      2  .write
      3  .option("path", "dbfs:/FileStore/tutorial/data")
      4  .saveAsTable("us_delay_flights_tbl"))

NameError: name 'flights_df' is not defined

Creando vistas

In [0]:
%sql
CREATE OR REPLACE GLOBAL TEMP VIEW vista_SFO AS
 SELECT fecha, delay, origin, destination from vuelos WHERE
 origin = 'SFO';


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW vista_JFK AS
 SELECT fecha, delay, origin, destination from vuelos WHERE
 origin = 'JFK'


In [0]:
df_sfo = spark.sql("SELECT fecha, delay, origin, destination FROM vuelos WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT fecha, delay, origin, destination FROM vuelos WHERE origin = 'JFK'")
# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("vista_SFO")
df_jfk.createOrReplaceTempView("vista_JFK")

In [0]:
%sql
SELECT * FROM global_temp.vista_SFO

fecha,delay,origin,destination
01011250,55,SFO,JFK
01012230,0,SFO,JFK
01010705,-7,SFO,JFK
01010620,-3,SFO,MIA
01010915,-3,SFO,LAX
01011005,-8,SFO,DFW
01011800,0,SFO,ORD
01011740,-7,SFO,LAX
01012015,-7,SFO,LAX
01012110,-1,SFO,MIA


In [0]:
spark.read.table("vista_JFK").show()


+--------+-----+------+-----------+
|   fecha|delay|origin|destination|
+--------+-----+------+-----------+
|01010900|   14|   JFK|        LAX|
|01011200|   -3|   JFK|        LAX|
|01011900|    2|   JFK|        LAX|
|01011700|   11|   JFK|        LAS|
|01010800|   -1|   JFK|        SFO|
|01011540|   -4|   JFK|        DFW|
|01011705|    5|   JFK|        SAN|
|01011530|   -3|   JFK|        SFO|
|01011630|   -3|   JFK|        SJU|
|01011345|    2|   JFK|        LAX|
|01011545|   -3|   JFK|        LAX|
|01011510|   -1|   JFK|        MIA|
|01011745|    7|   JFK|        SFO|
|01011250|    3|   JFK|        BOS|
|01011645|  142|   JFK|        LAX|
|01012135|   -2|   JFK|        LAX|
|01011715|   18|   JFK|        ORD|
|01011615|   25|   JFK|        IAH|
|01011850|   -2|   JFK|        SEA|
|01011725|   -5|   JFK|        BOS|
+--------+-----+------+-----------+
only showing top 20 rows



Drop vistas

In [0]:
%sql
DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view;
DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view


In [0]:
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")


Metadata

In [0]:
spark.catalog.listDatabases()


Out[22]: [Database(name='default', description='Default Hive database', locationUri='dbfs:/user/hive/warehouse')]

In [0]:
spark.catalog.listTables()


Out[23]: [Table(name='vista_jfk', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='vuelos', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
spark.catalog.listColumns("vista_jfk")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2944163253029737> in <module>
----> 1 spark.catalog.listColumns("vista_jfk")

/databricks/spark/python/pyspark/sql/catalog.py in listColumns(self, tableName, dbName)
    122         if dbName is None:
    123             dbName = self.currentDatabase()
--> 124         iter = self._jcatalog.listColumns(dbName, tableName).toLocalIterator()
    125         columns = []
    126         while iter.hasNext():

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception came fro

In [0]:
%sql
CACHE [LAZY] TABLE vuelos

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
no viable alternative at input 'CACHE ['(line 1, pos 6)

== SQL ==
CACHE [LAZY] TABLE vuelos
------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:298)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:159)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:88)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:106)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:77)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:97)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:74)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:791)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:151)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:791)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:958)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:788)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:605)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:28)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:26)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:60)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:60)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:582)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:615)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:607)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommandAndGetError(DriverWrapper.scala:526)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:561)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:431)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:374)
	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:225)
	at java.lang.Thread.run(Thread.java:748)

	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:130)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:605)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:28)
	at scala.util.Dynami

Tablas a dataframes

In [0]:
# In Python
us_flights_df = spark.sql("SELECT * FROM vuelos")
us_flights_df2 = spark.table("vuelos")


Data Sources for DataFrames and SQL Tables

DataFrameReader

In [0]:
%scala
file = """/databricks-datasets/learning-spark-v2/flights/summary-
 data/parquet/2010-summary.parquet"""
val df = spark.read.format("parquet").load(file)
// Use Parquet; you can omit format("parquet") if you wish as it's the default
val df2 = spark.read.load(file)
// Use CSV
val df3 = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .option("mode", "PERMISSIVE")
 .load("/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*")
// Use JSON
val df4 = spark.read.format("json")
 .load("/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")

It has a defined format and a recommended pattern for usage:
DataFrameReader.format(args).option("key", "value").schema(args).load()

https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html#manually-specifying-options


C:\Users\aitor.morais\Downloads\LearningSparkV2-master.zip\LearningSparkV2-master\databricks-datasets\learning-spark-v2

Parquet

In [0]:
# In Python
file = """/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/
 2010-summary.parquet/"""
df = spark.read.format("parquet").load(file)

Reading Parquet files into a Spark SQL table

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING parquet
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/
 2010-summary.parquet/" )


In [0]:
#Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

**TempView**  
The lifetime of this **temporary** table is tied to the SparkSession that was used to create this DataFrame.  
Generally, a session is an interaction between two or more entities.  In Spark 2.0 you can use SparkSession  
A SparkSession can be created without creating SparkConf, SparkContext or SQLContext, (they’re encapsulated within the SparkSession)  
The lifetime of this **Global** temporary view is tied to this Spark application.  
Spark application can be used:  
for a single batch job  
an interactive session with multiple jobs  
a long-lived server continually satisfying requests  
A Spark job can consist of more than just a single map and reduce.  
A Spark Application can consist of more than one session  
A SparkSession on the other hand is associated to a Spark Application

AVRO

In [0]:
%sql
-- In SQL 
CREATE OR REPLACE TEMPORARY VIEW episode_tbl
 USING avro
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
 )

In [0]:
#Python
spark.sql("SELECT * FROM episode_tbl").show(truncate=False)


Parquet

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING parquet
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/
 2010-summary.parquet/" )


In [0]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()


JSON

In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING json
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
 )


In [0]:
# In Python
file = "/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
df = spark.read.format("json").load(file)

CSV

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING csv
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*",
 header "true",
 inferSchema "true",
 mode "FAILFAST"
 )


In [0]:
file = "/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
df = (spark.read.format("csv")
 .option("header", "true")
 .schema(schema)
 .option("mode", "FAILFAST") # Exit if any errors
 .option("nullValue", "") # Replace any null data field with quotes
 .load(file))